In [1]:
import easyocr
from pdf2image import convert_from_path
import numpy as np
import re
from underthesea import word_tokenize, pos_tag

In [4]:
reader = easyocr.Reader(['vi', 'en'])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [39]:
result = reader.readtext('./images/test1.png', detail=True)

In [40]:
result

[([[75, 43], [329, 43], [329, 59], [75, 59]],
  'SOCIALIST REPUBLIC OF VIETNAM',
  0.7697532548557056),
 ([[468, 40], [573, 40], [573, 60], [468, 60]],
  'CỌNG HÒA XÃ',
  0.3857835571551236),
 ([[569, 37], [609, 37], [609, 73], [569, 73]], '#P!', 0.1792484630209864),
 ([[605, 41], [767, 41], [767, 59], [605, 59]],
  'CHU NGHIA VIẸT NAM',
  0.6244574226552123),
 ([[79, 57], [173, 57], [173, 73], [79, 73]],
  'Independence',
  0.9995456071953999),
 ([[181, 57], [243, 57], [243, 71], [181, 71]], 'Frccdom', 0.6449468996795203),
 ([[251, 53], [326, 53], [326, 77], [251, 77]],
  'Happiness',
  0.9880533548733477),
 ([[521, 55], [575, 55], [575, 73], [521, 73]], 'Độc lập', 0.8352117044274064),
 ([[635, 55], [711, 55], [711, 73], [635, 73]],
  'Hạnh phúc',
  0.8274330267076015),
 ([[153, 101], [249, 101], [249, 117], [153, 117]],
  'THE RECTOR',
  0.730179286517536),
 ([[567, 99], [673, 99], [673, 117], [567, 117]],
  'HIẸU TRƯỞNG',
  0.45255177509311334),
 ([[51, 121], [111, 121], [111, 137],

In [41]:
fields = []
coors = []

for r in result:
    txt = r[1].strip()
    if (txt.isnumeric()):
        continue
    txt = re.sub(r"['#@;:<>{}`+=~|.!?,]", "", txt)
    if (len(txt) == 0):
        continue
    if (r[2] < 0.3):
        continue
    coors.append(r[0])
    fields.append(txt)

In [42]:
f = open('./data/certificatePhrase.txt', 'r', encoding='utf-8')
lines = f.readlines()
phrases = []
for line in lines:
    if '\n' in line:
        phrases.append(line[:-1])
    else:
        phrases.append(line)

In [43]:
f = open('./data/vocab.txt', 'r', encoding='utf-8')
lines = f.readlines()
vocabs = []
for line in lines:
    if '\n' in line:
        vocabs.append(line[:-1])
    else:
        vocabs.append(line)

In [44]:
def find_similar(str1, str2):
    n = len(str1)
    cnt = 0
    for i in range(n):
        if str1[i] in str2:
            cnt += 1
    return cnt/n, cnt/len(str2)
    

In [45]:
s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
def remove_accents(input_str):
	s = ''
	for c in input_str:
		if c in s1:
			s += s0[s1.index(c)]
		else:
			s += c
	return s

In [46]:
def is_size_of_image(str):
    return str[0].isnumeric() and str[2].isnumeric() and str[1] == 'x'

In [47]:
def preprocess_word(str):
    #str = remove_accents(str)
    if (str.isnumeric()):
        return str
    if (is_size_of_image(str)):
        return str
    chatacters = [c for c in str if c.isalpha()]
    str = ''.join(chatacters)
    str = str.lower()
    return str

In [48]:
def find_similar(str1, str2):
    str1 = preprocess_word(str1)
    str2 = preprocess_word(str2)
    if str1 == str2:
        return 1, 1
    n = len(str1)
    if len(str1) > len(str2):
        return 0, 0
    if (n == 0 or len(str2) == 0):
        return 0, 0
    cnt = 0
    for i in range(n):
        if str1[i] == str2[i]:
            cnt += 1

    return cnt/n, cnt/len(str2)
    

In [49]:
def check_snake_case(str):
    if (len(str) == 0):
        return True
    if (str[0].isupper()):
        if (len(str) == 1):
            return True
        else:
            if(str[1].islower()):
                return True
    
    return False

In [50]:
def convert_snake_case(str):
    if (len(str) == 0):
        return str
    res = []
    res.append(str[0].upper())
    for i in range(1, len(str)):
        res.append(str[i].lower())

    return ''.join(res)

In [51]:
def check_word(str):
    sim_list = []
    for vocab in vocabs:
        sim1, sim2 = find_similar(str, vocab)
        sim_list.append((vocab, sim1, sim2))
    
    sim_list.sort(key = lambda x: (x[1], x[2]), reverse=True)
    if (str.isupper()):
        return sim_list[0][0].upper()
    if(str.islower()):
        return sim_list[0][0].lower()
    if (check_snake_case(str)):
        return convert_snake_case(sim_list[0][0])
    if sim_list[0][1] < 0.8:
        return ""
    return sim_list[0][0]

In [52]:
find_similar("5", '1')

(0.0, 0.0)

In [53]:
def tokenize(str):
    return word_tokenize(str)

In [54]:
init_fields = []
final_fields = []
final_coors = []

for i in range(len(fields)):
    field = fields[i]
    if len(field) < 2:
        continue
    words = field.split(" ")
    words = [check_word(word) for word in words]
    check = True
    for w in words:
        if (w != ''):
            check = False
            break
    if (check == False):
        init_fields.append(field)
        final_fields.append(' '.join(words))
        final_coors.append(coors[i])


In [55]:
for i in range(len(final_fields)):
    print(init_fields[i], "-", final_fields[i])

SOCIALIST REPUBLIC OF VIETNAM - SOCIALIST REPUBLIC OF VIETNAM
CỌNG HÒA XÃ - CỘNG HÒA XÃ
CHU NGHIA VIẸT NAM - CHU NGHIA LO VIỆT NAM
Independence - Independence
Frccdom - Freedom
Happiness - Happiness
Độc lập - Độc lập
Hạnh phúc - Hạnh phúc
THE RECTOR - THE RECTOR
HIẸU TRƯỞNG - HIỆU TRƯỞNG
OF VIET - OF VIETNAM
HUNG INDUSTRIAL UNIVERSITY - HUNG INDUSTRIAL UNIVERSITY
HUNG - HUNG
has conferred - has conferred
câp - cấp
THE DEGREE OF ENGINEER - THE DEGREE OF ENGINEERING
BẢNG KỸ SƯ - BẰNG KỸ SƯ
pon - ponl
Date of birth - Date of birth
Degree classification - Degree classification
Hà Nội ngày - Hà Nội ngày
Given under the seal of - Level independence the health of
Viet - Hung Industrial University - Vietnam & Hung Industrial University
HIẸU TRƯONG - HIỆU TRƯỞNG
Reg - Reg.no
thang - thang
o - of


In [56]:
check_word("NGHIA")

'NGHIA LO'